In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/delay-file-elec/final.xlsx


In [2]:
import wandb
from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()

my_secret = user_secrets.get_secret("wandb_api") 

wandb.login(key=my_secret)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
from datasets import load_dataset
import pandas as pd
# Load your dataset from a CSV file
dataset = pd.read_excel('/kaggle/input/delay-file-elec/final.xlsx',sheet_name='Final')

dataset=dataset[['Root Cause','Action Taken']]
dataset.head()

,Root Cause,Action Taken
0,Actually bar got twisted. It was not an electr...,We checked all HMDs before & after shear#2 but...
1,FM rectifier was tripped dueto external fault.,We reset the fault & restarted the mill.
2,Chain transfer motor-1 was tripped due to over...,We checked chain connected with this motor & t...
3,The gap in between Apron lifting proximity & i...,We just check tilting apron proximity & their ...
4,Tilting apron timeout alarm was the reason.,Operator reset the alarm & then restated the m...


In [4]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(dataset, test_size=0.2, random_state=42)

# Save to new CSV files for training and validation
train_df.to_csv('train.csv', index=False)
val_df.to_csv('val.csv', index=False)


In [5]:
from datasets import load_dataset

# Load the dataset from the CSV files you saved
dataset = load_dataset('csv', data_files={'train': 'train.csv', 'validation': 'val.csv'})
from datasets import load_dataset

data_files = {
    "train": "train.csv",
    "validation": "val.csv"
}

# Load the dataset from the CSV files
dataset = load_dataset('csv', data_files=data_files)


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [6]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained('t5-base')

def preprocess_function(examples):
    # Concatenate the root causes and actions into a single prompt-response format for T5
    inputs = ["root cause: " + str(cause) + " -> action:" for cause in examples['Root Cause']]
    targets = [str(action) for action in examples['Action Taken']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=512, truncation=True, padding="max_length")
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)



spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/640 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3935: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/161 [00:00<?, ? examples/s]

In [7]:
from transformers import T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained('t5-base')


model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [8]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results_t5',        # Directory where the results will be stored
    evaluation_strategy="epoch",      # Perform evaluation at the end of each epoch
    learning_rate=3e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,               # Only the last 3 models will be saved
    num_train_epochs=3,               # Total number of training epochs
    #predict_with_generate=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer
)


2024-05-01 02:32:34.101373: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-01 02:32:34.101465: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-01 02:32:34.226832: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_con

In [9]:
trainer.train()


wandb: Currently logged in as: ansafat (safawat). Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.16.6
wandb: Run data is saved locally in /kaggle/working/wandb/run-20240501_023242-gl2vno1d
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run visionary-snowflake-36
wandb: ⭐️ View project at https://wandb.ai/safawat/huggingface
wandb: 🚀 View run at https://wandb.ai/safawat/huggingface/runs/gl2vno1d
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss
1,No log,0.071041
2,No log,0.062245
3,No log,0.059349


TrainOutput(global_step=240, training_loss=0.2570006847381592, metrics={'train_runtime': 385.13, 'train_samples_per_second': 4.985, 'train_steps_per_second': 0.623, 'total_flos': 1169199149875200.0, 'train_loss': 0.2570006847381592, 'epoch': 3.0})

In [10]:
trainer.save_model('/kaggle/working/final_model')

In [11]:
# Save your model and tokenizer locally
model.save_pretrained('/Users/leo_mac/Downloads/delay/Electricaltroubleshooting_T5_FT')
tokenizer.save_pretrained('/Users/leo_mac/Downloads/delay/Electricaltroubleshooting_T5_FT')


('/Users/leo_mac/Downloads/delay/Electricaltroubleshooting_T5_FT/tokenizer_config.json',
 '/Users/leo_mac/Downloads/delay/Electricaltroubleshooting_T5_FT/special_tokens_map.json',
 '/Users/leo_mac/Downloads/delay/Electricaltroubleshooting_T5_FT/spiece.model',
 '/Users/leo_mac/Downloads/delay/Electricaltroubleshooting_T5_FT/added_tokens.json')

In [12]:
from transformers import pipeline

model_path = '/kaggle/working/final_model'
predictor = pipeline('text2text-generation', model=model_path, tokenizer=model_path)

# Test with an example root cause
test_root_cause = "drive tripped"
predicted_action = predictor(f"root cause: {test_root_cause} -> action:")
print(predicted_action[0]['generated_text'])


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


We reset the drive & then ok.
